In [1]:
import streamlit as st
from streamlit_chat import message
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
import os
import tempfile


In [2]:
uploaded_files=DirectoryLoader("./../data/").load()

ImportError: partition_docx is not available. Install the docx dependencies with pip install "unstructured[docx]"

In [ ]:

if uploaded_files:
        text = []
        for file in uploaded_files:
            file_extension = os.path.splitext(file.name)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file.read())
                temp_file_path = temp_file.name

            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
        text_chunks = text_splitter.split_documents(text)

In [20]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


[[ 6.76569268e-02  6.34958595e-02  4.87131663e-02  7.93049484e-02
   3.74480151e-02  2.65273103e-03  3.93748954e-02 -7.09838420e-03
   5.93614578e-02  3.15370075e-02  6.00980110e-02 -5.29051572e-02
   4.06067595e-02 -2.59308219e-02  2.98427958e-02  1.12688739e-03
   7.35148787e-02 -5.03818542e-02 -1.22386679e-01  2.37027854e-02
   2.97265742e-02  4.24768254e-02  2.56337989e-02  1.99515815e-03
  -5.69191836e-02 -2.71599442e-02 -3.29035930e-02  6.60248697e-02
   1.19007140e-01 -4.58791628e-02 -7.26214647e-02 -3.25841382e-02
   5.23413569e-02  4.50552553e-02  8.25307053e-03  3.67024280e-02
  -1.39414705e-02  6.53919429e-02 -2.64272261e-02  2.06371697e-04
  -1.36643583e-02 -3.62810344e-02 -1.95043348e-02 -2.89738476e-02
   3.94270569e-02 -8.84091258e-02  2.62428215e-03  1.36713954e-02
   4.83062603e-02 -3.11565734e-02 -1.17329232e-01 -5.11690266e-02
  -8.85287672e-02 -2.18963381e-02  1.42985675e-02  4.44168337e-02
  -1.34815322e-02  7.43392855e-02  2.66382322e-02 -1.98762473e-02
   1.79191